In [3]:
if 'already_executed' not in globals():
    already_executed = False

if not already_executed:
    import os
    os.chdir("../")
    %load_ext autoreload
    %autoreload 2
    already_executed = True

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from src.pricers.pricers import LSPISemiGradientPricer as LSPIPricer
from src.samplers.samplers import GeometricBrownianMotionPutSampler

In [12]:
sampler = GeometricBrownianMotionPutSampler(**{
    "asset0": 100.0,
    "sigma": 0.2,
    "r": 0.05,
    "strike": 100.0,
    "t": 1.0,
    "cnt_times": 365,
    "seed": None, 
    "cnt_trajectories": 1_000_0
})

In [13]:
pricer = LSPIPricer(sampler=sampler, **{
    "iterations": 1000,
    "lambda_reg": 0,
    "tol": 1e-9
})

In [14]:
pricer.price(test=False, quiet=False)

GBM sampling:   0%|          | 0/364 [00:00<?, ?it/s]

Iteration 0: ||Δw|| = 520.640801398
Iteration 1: ||Δw|| = 32.100551662
Iteration 2: ||Δw|| = 13.441596708
Iteration 3: ||Δw|| = 6.096826936
Iteration 4: ||Δw|| = 2.548319922
Iteration 5: ||Δw|| = 0.953443067
Iteration 6: ||Δw|| = 1.156776703
Iteration 7: ||Δw|| = 1.735432880
Iteration 8: ||Δw|| = 2.146858842
Iteration 9: ||Δw|| = 2.402443039
Iteration 10: ||Δw|| = 2.551440721
Iteration 11: ||Δw|| = 2.648237680
Iteration 12: ||Δw|| = 2.712566317
Iteration 13: ||Δw|| = 2.753880747
Iteration 14: ||Δw|| = 2.774284330
Iteration 15: ||Δw|| = 2.773745230
Iteration 16: ||Δw|| = 2.760575885
Iteration 17: ||Δw|| = 2.734155836
Iteration 18: ||Δw|| = 2.700323181
Iteration 19: ||Δw|| = 2.661411466
Iteration 20: ||Δw|| = 2.617293834
Iteration 21: ||Δw|| = 2.569466550
Iteration 22: ||Δw|| = 2.521286853
Iteration 23: ||Δw|| = 2.471901707
Iteration 24: ||Δw|| = 2.421206511
Iteration 25: ||Δw|| = 2.370370796
Iteration 26: ||Δw|| = 2.319200168
Iteration 27: ||Δw|| = 2.268223689
Iteration 28: ||Δw|| = 2.2

array([5.39806972])

In [15]:
K = sampler.strike
T = sampler.time_grid[-1]
r = -np.log(sampler.discount_factor[0, 1] / sampler.discount_factor[0, 0]) / sampler.time_deltas[0]
n_paths, n_times, _ = sampler.markov_state.shape
gamma = np.exp(-r * sampler.time_deltas[0])
S_paths = sampler.markov_state[:, :, 0]
time_grid_expanded = np.tile(sampler.time_grid, (n_paths, 1))
phi_all_raw = pricer._basis_functions_raw(S_paths, K, time_grid_expanded, T)
phi_all = pricer._scale_features(phi_all_raw, fit=True)

phi_curr = phi_all[:, :-1, :]  # Текущие состояния (t)
phi_next = phi_all[:, 1:, :]   # Следующие состояния (t+1)
payoff = sampler.payoff
payoff_next = sampler.payoff[:, 1:]  # Выплаты в t+1

# Выравнивание в 1D
n_features = phi_all.shape[-1]
phi_all_flat = phi_all.reshape(-1, n_features)
phi_curr_flat = phi_curr.reshape(-1, n_features)
phi_next_flat = phi_next.reshape(-1, n_features)
payoff_flat = payoff.reshape(-1)
payoff_next_flat = payoff_next.reshape(-1)

In [16]:
def bellman_opt_eq(phi_curr, phi_next, payoff_next, w, gamma, w_next=None):
    if w_next is None:
        w_next = w
    return ((
        phi_curr @ w - 
        gamma * np.max(
            np.concat([
                payoff_next[np.newaxis], 
                (phi_next @ w_next)[np.newaxis] \
                    * np.array(
                        (([1] * (n_times - 2)) + [0]) * n_paths
                    )
            ]), 
            axis=0
        )
    ) ** 2).mean()

In [17]:
bellman_opt_eq(
    phi_curr_flat, 
    phi_next_flat, 
    payoff_next_flat, 
    pricer.w, 
    gamma
)

np.float64(0.26417768710019124)

In [18]:
bellman_opt_eq(
    phi_curr_flat, 
    phi_next_flat, 
    payoff_next_flat, 
    pricer.w * 0.99, 
    gamma
)

np.float64(0.2603730748973656)

In [19]:
bellman_opt_eq(
    phi_curr_flat, 
    phi_next_flat, 
    payoff_next_flat, 
    pricer.w * 0.99, 
    gamma,
    pricer.w
)

np.float64(0.2738952623566142)

In [13]:
def price(phi_all, n_paths, payoff, w, gamma):
    pv_payoffs = np.zeros(n_paths)
    for p in range(n_paths):
        for t in range(n_times):
            phi_t = phi_all[p, t]
            Q_cont = phi_t @ w
            payoff_t = payoff[p, t]
            
            disc_factor = gamma**t
            if payoff_t >= Q_cont or t == n_times - 1:
                pv_payoffs[p] = disc_factor * payoff_t
                break
    return pv_payoffs.mean()

In [14]:
price(
    phi_all,
    n_paths, 
    sampler.payoff,
    pricer.w,
    gamma
)

np.float64(5.643907275958466)

In [59]:
price(
    phi_all,
    n_paths, 
    sampler.payoff,
    pricer.w * 0.99,
    gamma
)

np.float64(6.005314630688348)

In [5]:
import numpy as np
from numba import njit

@njit
def spread_with_gamma(arr, gamma):
    result = np.zeros_like(arr)
    rows, cols = np.where(np.abs(arr) > 1e-6)
    
    for i, j in zip(rows, cols):
        if arr[i, j] == 0:
            continue
            
        # Создаем диапазон для влияния
        k_values = np.arange(j + 1)
        powers = j - k_values
        influences = (gamma ** powers) * arr[i, j]
        
        # Находим позиции, которые еще не заполнены
        target_pos = k_values[result[i, k_values] == 0]
        result[i, target_pos] = influences[target_pos]
        
        # Обнуляем где power == 0
        result[i, j] = 0
        
    return result

In [9]:
a = np.array([
    [0, 0, 0, 1, 0, 0],
    [0,1,0,3,0,0]
])
gamma = 2

In [ ]:
spread_with_gamma(a, gamma)

In [3]:
from copy import deepcopy
import numpy as np
from typing import Optional
from sklearn.preprocessing import StandardScaler
from src.pricers.abstract_pricer import PricerAbstract
from src.samplers.abstract_sampler import SamplerAbstract


import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [21]:
from src.samplers.geometric_brownian_motion_put_sampler import GeometricBrownianMotionPutSampler
sampler = GeometricBrownianMotionPutSampler(**{
    "asset0": 100.0,
    "sigma": 0.2,
    "r": 0.05,
    "strike": 100.0,
    "t": 1.0,
    "cnt_times": 60,
    "seed": None, 
    "cnt_trajectories": 1_000_000
})

# LSPI

In [5]:
from dataclasses import dataclass
@dataclass 
class A:
    _: int

self = A(0)

def _create_features(self, markov_state: np.ndarray, time_grid: np.ndarray) -> np.ndarray:
    n_paths, n_times, state_dim = markov_state.shape
    T = time_grid[-1]
    
    # Нормализованное время до экспирации
    time_to_exp = (T - time_grid) / T
    time_to_exp = np.tile(time_to_exp, (n_paths, 1))[..., None]
    
    # Объединяем время и состояние
    features = np.concatenate([time_to_exp, markov_state], axis=-1)
    flat_features = features.reshape(-1, features.shape[-1])
    
    # Первый вызов - инициализируем и фитим трансформеры
    if not self.is_fitted_:
        self.scaler_ = StandardScaler()
        self.poly_ = PolynomialFeatures(degree=self.degree, include_bias=True)
        
        scaled = self.scaler_.fit_transform(flat_features)
        poly_features = self.poly_.fit_transform(scaled)
        self.n_features_ = poly_features.shape[1]
        self.is_fitted_ = True
    else:
        # Используем обученные трансформеры
        scaled = self.scaler_.transform(flat_features)
        poly_features = self.poly_.transform(scaled)
    
    return poly_features.reshape(n_paths, n_times, -1)

def _create_features(self, markov_state: np.ndarray, time_grid: np.ndarray) -> np.ndarray:
    M = markov_state[:, :, 0] / 100.
    exp_term = np.exp(-M / 2.0)
    T = time_grid[-1]
    t = time_grid

    phi = np.zeros((*markov_state[:, :, 0].shape, 7), dtype = float)
    phi[:, :, 0] = 1.0
    phi[:, :, 1] = exp_term
    phi[:, :, 2] = exp_term * (1.0 - M)
    phi[:, :, 3] = exp_term * (1.0 - 2.0 * M + 0.5 * M * M)
    phi[:, :, 4] = np.sin(np.pi * (T - t) / (2.0 * T))
    phi[:, :, 5] = np.log(np.maximum(T - t, 1e-10)) # 1e-10 чтобы не было log(0)
    phi[:, :, 6] = (t / T) ** 2

    scaler = StandardScaler()
    phi_reshaped = phi[:, :, 1:].reshape(-1, 6)
    phi_scaled = scaler.fit_transform(phi_reshaped)
    phi_scaled = phi_scaled.reshape(phi.shape[0], phi.shape[1], 6)


    phi_final = np.zeros_like(phi)
    phi_final[:, :, 0] = phi[:, :, 0]
    phi_final[:, :, 1:] = phi_scaled

    return phi_final

In [ ]:
self.sampler = sampler
self.sampler.sample()

In [27]:
r = -np.log(self.sampler.discount_factor[0, 1] / self.sampler.discount_factor[0, 0]) / (self.sampler.time_deltas[0])
self.dt = sampler.time_deltas[0]
self.is_fitted_ = False
self.degree = 3
self.iterations = 100
self.tol = 1e-6
self.reg_alpha = 1e-2
quiet = False

In [ ]:
# Параметры
n_paths, n_times, _ = self.sampler.markov_state.shape
gamma = np.exp(-r * self.dt)  # Коэффициент дисконтирования

# Создаем признаки
phi_all = _create_features(self,
    self.sampler.markov_state, 
    self.sampler.time_grid
)
n_features = phi_all.shape[-1]


w = np.zeros(n_features)

# Подготовка данных
phi_curr = phi_all[:, :-1, :]  # Текущие состояния (t)
phi_next = phi_all[:, 1:, :]    # Следующие состояния (t+1)
payoff_next = self.sampler.payoff[:, 1:]  # Выплаты в t+1

# Выравнивание в 1D
phi_curr_flat = phi_curr.reshape(-1, n_features)
phi_next_flat = phi_next.reshape(-1, n_features)
payoff_next_flat = payoff_next.reshape(-1)

# Флаг нетерминальных состояний
non_terminal = np.tile(
    np.arange(n_times-1, dtype=int) < n_times-2, 
    (n_paths, 1)
)
non_terminal_flat = non_terminal.reshape(-1)

# Итерации LSPI
for it in tqdm(range(self.iterations), desc="LSPI iterations"):
    prev_w = deepcopy(w)
    
    # Вычисляем Q-значения продолжения
    Q_cont_next = phi_next_flat @ w
    
    # Условие продолжения
    continue_cond = non_terminal_flat & (Q_cont_next >= payoff_next_flat)
    
    # Формируем систему уравнений
    diff_phi = phi_curr_flat - gamma * continue_cond[:, None] * phi_next_flat
    A = phi_curr_flat.T @ diff_phi
    b = gamma * phi_curr_flat.T @ ((~continue_cond) * payoff_next_flat)

    if self.reg_alpha is not None:
        A += np.eye(A.shape[0]) * self.reg_alpha
    w = np.linalg.solve(A, b)
    
    # Проверка сходимости
    diff_norm = np.linalg.norm(w - prev_w)
    if not quiet:
        print(f"Iteration {it}: ||Δw|| = {diff_norm:.9f}")
    if diff_norm < self.tol:
        if not quiet:
            print(f"Converged after {it} iterations")
        break

self.w = w

In [ ]:
pv_payoffs = np.zeros(n_paths)
for p in range(n_paths):
    for t in range(n_times):
        # Признаки текущего состояния
        phi_t = phi_all[p, t]
        Q_cont = phi_t @ w
        payoff_t = self.sampler.payoff[p, t]
        
        # Условие исполнения
        if payoff_t >= Q_cont or t == n_times - 1:
            disc_factor = self.sampler.discount_factor[p, t]
            pv_payoffs[p] = disc_factor * payoff_t
            break
mean = pv_payoffs.mean()

plt.figure(figsize=(10, 6))
plt.hist(pv_payoffs)
plt.axvline(mean)
plt.title("Train")
plt.grid()
plt.show()
mean

In [ ]:
self.sampler.sample()

In [ ]:
phi_all = _create_features(self,
    self.sampler.markov_state, 
    self.sampler.time_grid
)
pv_payoffs = np.zeros(n_paths)
for p in range(n_paths):
    for t in range(n_times):
        # Признаки текущего состояния
        phi_t = phi_all[p, t]
        Q_cont = phi_t @ w
        payoff_t = self.sampler.payoff[p, t]
        
        # Условие исполнения
        if payoff_t >= Q_cont or t == n_times - 1:
            disc_factor = self.sampler.discount_factor[p, t]
            pv_payoffs[p] = disc_factor * payoff_t
            break
mean = pv_payoffs.mean()

plt.figure(figsize=(10, 6))
plt.hist(pv_payoffs)
plt.axvline(mean)
plt.title("Test")
plt.grid()
plt.show()
mean

In [ ]:
n_traj = 10

plt.figure(figsize=(10, 6))
plt.plot(self.sampler.time_grid, self.sampler.payoff[:n_traj, :].T)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(self.sampler.time_grid, (self.sampler.payoff[:, :] * self.sampler.discount_factor)[:n_traj, :].T)
plt.show()

# LSMC

In [ ]:
from src.pricers.american_monte_carlo import AmericanMonteCarloPricer
pricer = AmericanMonteCarloPricer(sampler, degree=3)
pricer.price();
pricer.result

In [ ]:
import scipy
exercise_boundary_lsmc = []
for i, t in tqdm(list(
    enumerate(sampler.time_grid)
)):
    if pricer.weights[i] is None:
        print("None")
        exercise_boundary_lsmc.append(None)
        continue

    func_to_find_root = lambda x: (
        sampler.discount_factor[0, i] * max(0, 100. - x) - 
        (
            pricer.basis_functions_transformer.transform(
                pricer.scalers[i].transform([[x]])
            ) @ pricer.weights[i]
        ).reshape(-1)[0]
    )

    root = scipy.optimize.minimize_scalar(
        lambda x: func_to_find_root(x)**2,
        bounds=[np.percentile(sampler.markov_state[:, i, 0], 0.99), 100]
    )["x"]
    
    if abs(func_to_find_root(root)) < 1e-6:
        exercise_boundary_lsmc.append(
            root
        )
    else:
        exercise_boundary_lsmc.append(
            None
        )

    payoff = np.vectorize(lambda x: sampler.discount_factor[0, i] * max(0, 100. - x))
    continuation_value = np.vectorize(lambda x: (
        pricer.basis_functions_transformer.transform(
            pricer.scalers[i].transform([[x]])
        ) @ pricer.weights[i]
    ).reshape(-1)[0])
    plt.figure(figsize=(10, 6))
    plt.title(f"Continuation value at t={t}")
    grid = np.linspace(np.percentile(sampler.markov_state[:, i, 0], 0.99), 100, 30)
    plt.plot(grid, continuation_value(grid), label="CV")
    plt.plot(grid, payoff(grid), label="Payoff")
    plt.legend()
    plt.grid()
    plt.show()

plt.figure(figsize=(10, 6))
plt.plot(sampler.time_grid, exercise_boundary_lsmc, "-o", markersize=2)
plt.show()

In [ ]:
exercise_boundary_lsmc

In [ ]:
sampler.discount_factor[0, i]

In [ ]:
exercise_boundary_lsmc

In [15]:
# import scipy
# exercise_boundary_lsmc = []
# for i, t in tqdm(list(enumerate(range(len(sampler.time_grid))))):
#     if pricer.weights[i] is None:
#         exercise_boundary_lsmc.append(None)
#         continue
#     exercise_boundary_lsmc.append(
#         scipy.optimize.minimize_scalar(
#             lambda x: (
#                 sampler.discount_factor[0, i] * max(0, 100. - x) - 
#                 (
#                     pricer.basis_functions_transformer.transform([[x]]) @ pricer.weights[i]
#                 ).reshape(-1)[0]
#             )**2,
#             bounds=[0, 100]
#         )["x"]
#     )

# plt.figure(figsize=(10, 6))
# plt.plot(sampler.time_grid, exercise_boundary_lsmc)
# plt.show()

In [ ]:
pricer.weights